In [28]:
from pyteomics import mzxml, auxiliary
from matplotlib import pyplot as plt
import numpy as np 
import os
import pandas as pd
import math
import random
from itertools import product

f = "/home/rcardiff/meyer_raw_data/C18_neg/CL/CL1.mzXML"
#f = '/home/rcardiff/ryan/ryan/meyer_raw_data/C18_neg/CL/CL7.mzXML'
a = mzxml.read(f) 

rt, mz, intensity = ([] for i in range(3))

for spectra in a:
	mz.append(spectra.get('m/z array'))
	intensity.append(spectra.get('intensity array'))
	rt.append(spectra.get('retentionTime'))


minMz = 0
maxMz = 1000
minRt = 0
maxRt = 30


#rt_buckets = np.linspace(.02,.25,5)
#mz_buckets = np.linspace(.001, .01, 5)

In [2]:
from pyteomics import mzxml, auxiliary
from matplotlib import pyplot as plt
import numpy as np 
import os
import pandas as pd
import math
import random
from itertools import product

directory = "/home/rcardiff/ryan/ryan/meyer_raw_data/C18_neg/CL/"
os.chdir(directory)
files = os.listdir(directory)

rt, mz, intensity = ([] for i in range(3))
minMz = 0
maxMz = 1000
minRt = 0
maxRt = 30

for f in files:
    if f.endswith(".mzXML"):
        a = mzxml.read(f) 
        for spectra in a:
            mz.append(spectra.get('m/z array'))
            intensity.append(spectra.get('intensity array'))
            rt.append(spectra.get('retentionTime'))


In [29]:
mz_out = np.concatenate(mz).ravel()


rt_out = []
n = len(mz)
for i in range(n):
    rt_out.append([rt[i]]*len(mz[i]))
rt_out = np.concatenate(rt_out).ravel()
i_out = np.concatenate(intensity).ravel()

print(len(i_out))
print(len(mz_out))
print(len(rt_out))

#rounding is very slow - is there a faster way?
mz_round = [round(m1*(1/.001))/(1/.001) for m1 in mz_out]
mz_round = ([np.round(m,3) for m in mz_round])
mz_out = mz_round

rt_round = ([round(r*(1/.02))/(1/.02) for r in rt_out])   
rt_out = rt_round

print(len(mz_out))
print(len(rt_out))

1608111
1608111
1608111
1608111
1608111


In [30]:
#rt_list = np.arange(minRt, maxRt+.001, .1)
#rt_list = [np.round(r,3) for r in rt_list]
mz_list = np.arange(minMz,maxMz, .001)
mz_list = np.round(mz_list,3)

zipped_data = list(zip(mz_out, rt_out, i_out))
zipped_data = sorted(zipped_data, key = lambda tup : tup[0])
from tqdm import tqdm_notebook as tqdm

In [25]:
print(zipped_data[3][0])
print(mz_list[70000-16])

69.985
69.984


In [ ]:
#NOT THIS ONE
import csv

print(len(mz_out))
#min_mz_out = min(mz_out)
min_mz_out = 150
#max_mz_out = max(mz_out)
max_mz_out = 250
with open('/home/rcardiff/combined_test2_highres.csv', 'w') as f:
    writer = csv.writer(f, delimiter=',')
    #writer.writerow(rt_list)
    for mz in tqdm(mz_list):
        col = np.zeros(int(30/.02)+1)
        if mz < min_mz_out:
            writer.writerow(col)
            continue
        if mz > max_mz_out:
            writer.writerow(col)
            continue
            
        indices = [i for i, x in enumerate(mz_out) if x == mz]
        rt_i = [rt_out[i] for i in indices] 
        i_i = [i_out[i] for i in indices]

        for i in rt_i:
            col[int(i/.02)] += i_i[rt_i.index(i)]
        mz_out = np.delete(mz_out,indices)
        rt_out = np.delete(rt_out,indices)
        i_out = np.delete(i_out,indices)
        writer.writerow(col)
print(len(mz_out))

In [31]:
#YES THIS ONE
import csv

print(len(mz_out))
#min_mz_out = min(mz_out)
min_mz_out = 150
#max_mz_out = max(mz_out)
max_mz_out = 250

mz_loc = 0
with open('/home/rcardiff/high_res_test/neg_CL6.csv', 'w') as f:
    writer = csv.writer(f, delimiter=',')
    for mz in tqdm(mz_list):
        col = np.zeros(int(30/.02)+1)
        if mz < min_mz_out:
            #writer.writerow(col)
            continue
        if mz > max_mz_out:
            #writer.writerow(col)
            continue
            
        while zipped_data[mz_loc][0] <= mz:
            rt = zipped_data[mz_loc][1]
            i = zipped_data[mz_loc][2]
            col[int(rt/.02)] += i
            mz_loc += 1
        writer.writerow(col)
print("Done")

1608111



Done


In [31]:
print(mz_loc)

7437888


In [ ]:
#not this one
indices = [i for i, x in enumerate(mz_out) if x == mz]
#         if len(indices) > 0:
#             break
rt_i = [rt_out[i] for i in indices] 
i_i = [i_out[i] for i in indices]

col = np.zeros(int(30/.02)+1)
for i in rt_i:
    col[int(i/.02)] += i_i[rt_i.index(i)]
mz_out = np.delete(mz_out,indices)
rt_out = np.delete(rt_out,indices)
i_out = np.delete(i_out,indices)
writer.writerow(col)

In [32]:
#run time ~2 min 
#https://stackoverflow.com/questions/902761/saving-a-numpy-array-as-an-image
from numpy import genfromtxt
import numpy as np
import os
#from PIL import Image
#import i"mageio
#import matplotlib.image
import cv2

f = "/home/rcardiff/high_res_test/neg_CL6.csv"

image_array = genfromtxt(f, delimiter=',')
image_array = image_array.astype('float64')

cv2.imwrite("/home/rcardiff/high_res_test/neg_CL7.png", image_array)

True

In [ ]:
#image_array /= np.max(image_array)
#image_array *= 255
#img = Image.fromarray(image_array, 'L')
#img.save('/home/rcardiff/grid_test3.png')


#no image
#imageio.imwrite('outfile.jpg', image_array)

#no features
#matplotlib.image.imsave('name.png', image_array)



In [ ]:
print(len(image_array[0]))
print(len(image_array))

def first_nonzero(arr, axis, invalid_val=-1):
    mask = arr!=0
    return np.where(mask.any(axis=axis), mask.argmax(axis=axis), invalid_val)

d = first_nonzero(image_array, axis=1, invalid_val=0)
np.set_printoptions(threshold=np.inf)
print(np.nonzero(d)[0][0])

In [ ]:
#run time ~1min for 1500x80000 image
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from skimage.measure import regionprops
import matplotlib.patches as mpatches
from skimage.morphology import label
import pandas as pd

image = cv.imread('/home/rcardiff/combined_img.png',0)

edges = feature.canny(image, sigma=1.5,
                     low_threshold=193,
                     high_threshold=225)


# Label image regions.
label_image = label(edges)
print(len(regionprops(label_image)))




In [ ]:
#run time ~1min for 1500x80000 image
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature
from skimage.measure import regionprops
import matplotlib.patches as mpatches
from skimage.morphology import label
import pandas as pd

image = cv.imread('/home/rcardiff/combined_img.png',0)
image = image[10000:11000]
edges = feature.canny(image, sigma=.5,
                     low_threshold=193,
                     high_threshold=225)


# Label image regions.
label_image = label(edges)
print(len(regionprops(label_image)))

fig, ax5 = plt.subplots(figsize=(14, 14))
ax5.imshow(image, cmap=plt.cm.gray)
ax5.axis('off')

for region in regionprops(label_image):
    # Draw rectangle around segmented coins.
    minr, minc, maxr, maxc = region.bbox
    rect = mpatches.Rectangle((minc, minr),
                              maxc - minc,
                              maxr - minr,
                              fill=False,
                              edgecolor='red',
                              linewidth=1)
    ax5.add_patch(rect)
    
plt.tight_layout()
plt.show()

In [ ]:
#run time <1 min
data = {'Feature':[],'min_mz':[],'max_mz':[],'mean_mz':[],'min_rt':[],'max_rt':[],'mean_rt':[]}
df = pd.DataFrame(data)
c = 0
for region in regionprops(label_image):
    min_mz, min_rt, max_mz, max_rt = region.bbox
    r = pd.Series(['FT'+str(c), (min_mz/100), (max_mz/100), (np.mean([min_mz, max_mz])/100), min_rt*1.2, max_rt*1.2, np.mean([min_rt, max_rt])*1.2], index = ['Feature','min_mz','max_mz','mean_mz','min_rt','max_rt','mean_rt'])
    df = df.append(r,ignore_index = True)
    c+=1
df.to_csv('/home/rcardiff/feature_test_1217-2.tsv', sep = '\t')

In [ ]:
image = cv.imread('/home/rcardiff/combined_img.png',0)
fig, ax = plt.subplots(1, 1)
ax.hist(image.ravel(), bins=8, range=[0, 256]) #ravel returns flattened 1d array
ax.set_xlim(32, 193);
ax.set_ylim(0,10)